In [678]:
print("Hello world")

Hello world


In [26]:
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
!pip install langchain-together

  Obtaining dependency information for langchain-together from https://files.pythonhosted.org/packages/a8/69/b3cbcf5b43acbc098c012ef75035fb0dc1e0f227f5161329ef9884a25ba4/langchain_together-0.3.0-py3-none-any.whl.metadata
  Using cached langchain_together-0.3.0-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for langchain-openai<0.4,>=0.3 from https://files.pythonhosted.org/packages/aa/31/1f0baf6490b082bf4d06f355c5e9c28728931dbf321f3ca03137617a692e/langchain_openai-0.3.27-py3-none-any.whl.metadata
  Obtaining dependency information for openai<2.0.0,>=1.86.0 from https://files.pythonhosted.org/packages/64/46/a10d9df4673df56f71201d129ba1cb19eaff3366d08c8664d61a7df52e65/openai-1.93.0-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken<1,>=0.7 from https://files.pythonhosted.org/packages/6f/07/c67ad1724b8e14e2b4c8cca04b15da158733ac60136879131db05dda7c30/tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tiktoken-0.9.0-cp311-cp311-win


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [679]:
from google.oauth2 import service_account
from langchain_google_community import CalendarToolkit

#Path of service account JSON key
SERVICE_ACCOUNT_FILE = "service-account.json"

#scope of calender access
SCOPES = ["https://www.googleapis.com/auth/calendar"]

#Test Calender ID
CALENDAR_ID = "60283bb03155968145ad69adbdb9891ab54720ff7509b44685ec088112ab5bb2@group.calendar.google.com"

# create credential using service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE,scopes = SCOPES
)

In [680]:
from googleapiclient.discovery import build

# build teh api resource manually
api_resource = build('calendar', 'v3', credentials=credentials)

# initialize the toolkit with the api_resource
toolkit = CalendarToolkit(api_resource=api_resource)

In [681]:
tools = toolkit.get_tools() # get all tools
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 CalendarSearchEvents(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 CalendarUpdateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 GetCalendarsInfo(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 CalendarMoveEvent(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 CalendarDeleteEvent(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>),
 GetCurrentDatetime(api_resource=<googleapiclient.discovery.Resource object at 0x00000292BFAD4810>)]

In [682]:
create_event_tool = CalendarCreateEvent(api_resource = api_resource)

In [683]:
# Invoke with event data
result = create_event_tool.invoke({
    "calendar_id": CALENDAR_ID,
    "summary": "Test Event",
    "start_datetime": "2025-07-08 10:00:00",
    "end_datetime": "2025-07-08 10:30:00",
    "timezone": "Asia/Kolkata",
    "description": "Created via service account"
})

print(result)

Event created: https://www.google.com/calendar/event?eid=bTAxcDJvcGJkdHRkc2owcTJvbzdmcjcyYzQgNjAyODNiYjAzMTU1OTY4MTQ1YWQ2OWFkYmRiOTg5MWFiNTQ3MjBmZjc1MDliNDQ2ODVlYzA4ODExMmFiNWJiMkBn


In [684]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can access the API key
api_key = os.getenv("TOGETHER_API_KEY")
# print(api_key)  # Optional: just to verify

In [685]:


from langchain_google_community.calendar.create_event import CalendarCreateEvent
from langchain_google_community.calendar.search_events import CalendarSearchEvents
from langchain_google_community.calendar.update_event import CalendarUpdateEvent
from langchain_google_community.calendar.delete_event import CalendarDeleteEvent
from langchain_google_community.calendar.move_event import CalendarMoveEvent



In [686]:
from langchain_core.tools import tool

In [687]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

SERVICE_ACCOUNT_FILE = 'service-account.json'
SCOPES = ['https://www.googleapis.com/auth/calendar']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
calendar_service = build('calendar', 'v3', credentials=credentials)


In [688]:
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional, Type

class GoogleCalendarBaseTool:
    """Base class for Google Calendar tools using a service account."""
    api_resource = None

    def __init__(self, api_resource):
        self.api_resource = api_resource

    @classmethod
    def from_api_resource(cls, api_resource):
        return cls(api_resource=api_resource)


In [689]:
from dateutil import parser, tz
from datetime import datetime

class GetEventsSchema(BaseModel):
    start_datetime: str = Field(..., description="Start datetime (YYYY-MM-DDTHH:MM:SS)")
    end_datetime: str = Field(..., description="End datetime (YYYY-MM-DDTHH:MM:SS)")
    max_results: int = Field(default=10, description="Max results to return")
    timezone: str = Field(default="Asia/Kolkata", description="Timezone (TZ database name)")

class ListGoogleCalendarEvents(GoogleCalendarBaseTool):
    def _parse_event(self, event, timezone):
        start = event['start'].get('dateTime', event['start'].get('date'))
        start = parser.parse(start).astimezone(tz.gettz(timezone)).strftime('%Y/%m/%d %H:%M:%S')
        end = event['end'].get('dateTime', event['end'].get('date'))
        end = parser.parse(end).astimezone(tz.gettz(timezone)).strftime('%Y/%m/%d %H:%M:%S')
        event_parsed = dict(start=start, end=end)
        for field in ['summary', 'description', 'location', 'hangoutLink']:
            event_parsed[field] = event.get(field, None)
        # ADD THIS LINE:
        event_parsed['id'] = event.get('id')
        return event_parsed


    def _run(self, start_datetime, end_datetime, max_results=10, timezone="Asia/Kolkata"):
        calendar_id = "60283bb03155968145ad69adbdb9891ab54720ff7509b44685ec088112ab5bb2@group.calendar.google.com"
        events = []
        start = datetime.strptime(start_datetime, '%Y-%m-%dT%H:%M:%S')
        start = start.replace(tzinfo=tz.gettz(timezone)).isoformat()
        end = datetime.strptime(end_datetime, '%Y-%m-%dT%H:%M:%S')
        end = end.replace(tzinfo=tz.gettz(timezone)).isoformat()
        events_result = self.api_resource.events().list(
            calendarId=calendar_id,
            timeMin=start,
            timeMax=end,
            maxResults=max_results,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        cal_events = events_result.get('items', [])
        events.extend(cal_events)
        events = sorted(events, key=lambda x: x['start'].get('dateTime', x['start'].get('date')))
        return [self._parse_event(e, timezone) for e in events]


In [690]:
class CreateEventSchema(BaseModel):
    start_datetime: str = Field(..., description="Start datetime (YYYY-MM-DDTHH:MM:SS)")
    end_datetime: str = Field(..., description="End datetime (YYYY-MM-DDTHH:MM:SS)")
    summary: str = Field(..., description="Event title")
    location: Optional[str] = Field(default="", description="Event location")
    description: Optional[str] = Field(default="", description="Event description")
    timezone: str = Field(default="Asia/Kolkata", description="Timezone (TZ database name)")

class CreateGoogleCalendarEvent(GoogleCalendarBaseTool):
    def _run(self, start_datetime, end_datetime, summary, location="", description="", timezone="Asia/Kolkata"):
        calendar_id = "60283bb03155968145ad69adbdb9891ab54720ff7509b44685ec088112ab5bb2@group.calendar.google.com"
        start = datetime.strptime(start_datetime, '%Y-%m-%dT%H:%M:%S')
        start = start.replace(tzinfo=tz.gettz(timezone)).isoformat()
        end = datetime.strptime(end_datetime, '%Y-%m-%dT%H:%M:%S')
        end = end.replace(tzinfo=tz.gettz(timezone)).isoformat()
        body = {
            'summary': summary,
            'start': {'dateTime': start},
            'end': {'dateTime': end}
        }
        if location:
            body['location'] = location
        if description:
            body['description'] = description
        event = self.api_resource.events().insert(calendarId=calendar_id, body=body).execute()
        return "Event created: " + event.get('htmlLink', 'Failed to create event')


In [691]:
from pydantic import BaseModel, Field
from typing import Optional

class DeleteEventSchema(BaseModel):
    event_id: str = Field(..., description="The event ID to delete.")
    calendar_id: Optional[str] = Field(
        default=None, description="The calendar ID. Defaults to your test calendar."
    )

class DeleteGoogleCalendarEvent:
    """
    Delete an event from your Google Calendar.

    - Requires only the event ID.
    - Always uses your test calendar ID unless another is specified.
    """

    def __init__(self, api_resource):
        self.api_resource = api_resource
        self.calendar_id = "60283bb03155968145ad69adbdb9891ab54720ff7509b44685ec088112ab5bb2@group.calendar.google.com"

    def _run(self, event_id, calendar_id=None):
        calendar_id = calendar_id or self.calendar_id
        try:
            self.api_resource.events().delete(
                calendarId=calendar_id,
                eventId=event_id
            ).execute()
            return f"Event {event_id} deleted from calendar {calendar_id}."
        except Exception as e:
            return f"Failed to delete event: {e}"


In [692]:
from dateutil import parser, tz
from datetime import datetime

class PostponeEventSchema(BaseModel):
    event_id: str = Field(..., description="The event ID to postpone.")
    new_start_datetime: str = Field(..., description="New start datetime (YYYY-MM-DDTHH:MM:SS)")
    new_end_datetime: str = Field(..., description="New end datetime (YYYY-MM-DDTHH:MM:SS)")
    timezone: str = Field(default="Asia/Kolkata", description="Timezone (TZ database name)")
    calendar_id: Optional[str] = Field(
        default=None, description="The calendar ID. Defaults to your test calendar."
    )

class PostponeGoogleCalendarEvent:
    """
    Postpone (reschedule) an event in your Google Calendar.

    - Requires event ID and new start/end datetimes.
    - Always uses your test calendar ID unless another is specified.
    """

    def __init__(self, api_resource):
        self.api_resource = api_resource
        self.calendar_id = "60283bb03155968145ad69adbdb9891ab54720ff7509b44685ec088112ab5bb2@group.calendar.google.com"

    def _run(self, event_id, new_start_datetime, new_end_datetime, timezone="Asia/Kolkata", calendar_id=None):
        calendar_id = calendar_id or self.calendar_id
        try:
            print("this is inside the class and -----")
            print(event_id)
            # Get the existing event
            event = self.api_resource.events().get(
                calendarId=calendar_id,
                eventId=event_id
            ).execute()

            # Format new times
            start = datetime.strptime(new_start_datetime, '%Y-%m-%dT%H:%M:%S').replace(tzinfo=tz.gettz(timezone)).isoformat()
            end = datetime.strptime(new_end_datetime, '%Y-%m-%dT%H:%M:%S').replace(tzinfo=tz.gettz(timezone)).isoformat()

            event['start']['dateTime'] = start
            event['end']['dateTime'] = end
            event['start']['timeZone'] = timezone
            event['end']['timeZone'] = timezone

            updated_event = self.api_resource.events().update(
                calendarId=calendar_id,
                eventId=event_id,
                body=event
            ).execute()

            return f"Event postponed: {updated_event.get('htmlLink', 'No link')}"
        except Exception as e:
            return f"Failed to postpone event: {e}"


In [693]:
# Initialize tools
list_events_tool = ListGoogleCalendarEvents.from_api_resource(calendar_service)
create_event_tool = CreateGoogleCalendarEvent.from_api_resource(calendar_service)

# List events
events = list_events_tool._run(
    start_datetime="2025-07-06T00:00:00",
    end_datetime="2025-07-09T23:59:59",
    max_results=10,
    timezone="Asia/Kolkata"
)
for e in events:
    print(e['start'], e['summary'])

# Create an event
result = create_event_tool._run(
    start_datetime="2025-07-10T10:00:00",
    end_datetime="2025-07-10T11:00:00",
    summary="AI Project Meeting",
    location="Online",
    description="Discuss project milestones",
    timezone="Asia/Kolkata"
)
print(result)


2025/07/07 05:00:00 an event
2025/07/08 10:00:00 Test Event
2025/07/08 10:00:00 Test Event
2025/07/08 10:00:00 Test Event
2025/07/08 10:00:00 Test Event
2025/07/08 10:00:00 Test Event
2025/07/08 10:00:00 Test Event
2025/07/09 12:00:00 Team Sync
2025/07/09 12:00:00 Team Sync
2025/07/09 12:00:00 Team Sync
Event created: https://www.google.com/calendar/event?eid=aGNxNWtyZjRpZDg0OXE3ZDRtc3Z0ZzV0NmMgNjAyODNiYjAzMTU1OTY4MTQ1YWQ2OWFkYmRiOTg5MWFiNTQ3MjBmZjc1MDliNDQ2ODVlYzA4ODExMmFiNWJiMkBn


In [719]:
from langchain.tools import tool

In [720]:
@tool
def create_event_tool(start_datetime, end_datetime, summary, location="", description="", ):
    """
    Create a Google Calendar event.

    Args:
        start_datetime (str): Start datetime (YYYY-MM-DDTHH:MM:SS).
        end_datetime (str): End datetime (YYYY-MM-DDTHH:MM:SS).
        summary (str): Event title.
        location (str, optional): Event location.
        description (str, optional): Event description.
        timezone (str): Timezone.

    Returns:
        str: Confirmation message with event link.
    """
    timezone="Asia/Kolkata"
    tool = CreateGoogleCalendarEvent(api_resource)
    return tool._run(
        start_datetime=start_datetime,
        end_datetime=end_datetime,
        summary=summary,
        location=location,
        description=description,
        timezone=timezone
    )


In [696]:
@tool
def list_events_tool(start_datetime, end_datetime, max_results=10,):
    """
    List Google Calendar events in a date range.

    Args:
        start_datetime (str): Start datetime (YYYY-MM-DDTHH:MM:SS).
        end_datetime (str): End datetime (YYYY-MM-DDTHH:MM:SS).
        max_results (int): Maximum results to return.
        timezone (str): Timezone.

    Returns:
        list: List of event dicts (each includes event ID, summary, times, etc.).
    """
    timezone="Asia/Kolkata"
    tool = ListGoogleCalendarEvents(api_resource)
    return tool._run(
        start_datetime=start_datetime,
        end_datetime=end_datetime,
        max_results=max_results,
        timezone=timezone
    )


In [697]:
from typing import TypedDict, cast
from langchain.tools import tool

@tool
def postpone_event_tool(
    start_datetime: str,
    end_datetime: str,
    user_query: str,
    new_start_datetime: str,
    new_end_datetime: str,
) -> str:
    """
    Postpone a Google Calendar event based on a natural language user query.
    Uses an LLM to select the correct event if the reference is ambiguous.

    Args:
        start_datetime (str): Start datetime for search window.
        end_datetime (str): End datetime for search window.
        user_query (str): The user's original query or meeting description.
        new_start_datetime (str): New start datetime for the event.
        new_end_datetime (str): New end datetime for the event.
        llm: An LLM instance capable of function-calling or text completion.

    Returns:
        str: Confirmation message or clarification prompt.
    """ 
    timezone = "Asia/Kolkata"
    args = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime
    }
    events = list_events_tool.invoke(args)
    if not events:
        return "No events found in the specified time window."

    # Prepare event options for the LLM
    event_options = [
        f"{idx+1}. {e.get('summary', 'No Title')} at {e.get('start')} (ID: {e.get('id')})"
        for idx, e in enumerate(events)
    ]
    options_text = "\n".join(event_options)


    # Compose LLM prompt
    prompt = (
        f"User query: '{user_query}'\n"
        f"Here are the events found:\n{options_text}\n"
        "Based on the user's query, which event ID best matches the intent? "
        "Just reply with the event ID."
    )

    # Call the LLM to select the best event
    class output(TypedDict):
        event_ids = list[str]

    llm_response = cast(output,llm.with_structured_output(output).invoke(prompt))
    selected_event_id = llm_response

    # Find the event by ID
    event = next((e for e in events if e.get('id') in selected_event_id), None)
    if not event:
        return (
            "Sorry, I couldn't confidently match your request to a specific event. "
            "Please clarify or specify the event ID."
        )

    # Postpone the event
    tool = PostponeGoogleCalendarEvent(api_resource)
    result = tool._run(
        event_id=str(event.get('id')),
        new_start_datetime=new_start_datetime,
        new_end_datetime=new_end_datetime,
        timezone=timezone
    )
    return f"Postponed event: {event.get('summary')} ({selected_event_id}): {result}"


In [698]:
from typing import TypedDict, cast
from langchain.tools import tool

@tool
def delete_event_tool(
    start_datetime: str,
    end_datetime: str,
    user_query: str,
) -> str:
    """
    Delete a Google Calendar event based on a natural language user query.
    Uses an LLM to select the correct event if the reference is ambiguous.

    Args:
        start_datetime (str): Start datetime for search window.
        end_datetime (str): End datetime for search window.
        user_query (str): The user's original query or meeting description.
        llm: An LLM instance capable of function-calling or text completion.

    Returns:
        str: Confirmation message or clarification prompt.
    """
    timezone = "Asia/Kolkata"
    args = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime
    }
    events = list_events_tool.invoke(args)
    if not events:
        return "No events found in the specified time window."

    # Prepare event options for the LLM
    event_options = [
        f"{idx+1}. {e.get('summary', 'No Title')} at {e.get('start')} (ID: {e.get('id')})"
        for idx, e in enumerate(events)
    ]
    options_text = "\n".join(event_options)

    # Compose LLM prompt
    prompt = (
        f"User query: '{user_query}'\n"
        f"Here are the events found:\n{options_text}\n"
        "Based on the user's query, which event ID best matches the intent for deletion? "
        "Just reply with the event ID."
    )

    # Call the LLM to select the best event
    class output(TypedDict):
        event_ids = list[str]

    llm_response = cast(output, llm.with_structured_output(output).invoke(prompt))
    selected_event_id = llm_response

    # Find the event by ID
    event = next((e for e in events if e.get('id') in selected_event_id), None)
    if not event:
        return (
            "Sorry, I couldn't confidently match your request to a specific event. "
            "Please clarify or specify the event ID."
        )

    # Delete the event
    tool = DeleteGoogleCalendarEvent(api_resource)
    result = tool._run(
        event_id=str(event.get('id')),
        calendar_id=None,  # Will default to primary or your pre-set calendar ID
    )
    return f"Deleted event: {event.get('summary')} ({selected_event_id}): {result}"


In [699]:
from langgraph.prebuilt import ToolNode
tool_node = ToolNode([create_event_tool,list_events_tool,postpone_event_tool,delete_event_tool])

In [700]:
from langgraph.prebuilt import ToolNode

calendar_tools = [
    create_event_tool,
    list_events_tool,
    postpone_event_tool,
    delete_event_tool
]

tool_node = ToolNode(calendar_tools)


In [701]:
llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
)

# Use the 'tools' list, which contains properly formatted LangChain tools
model_with_tools = llm.bind_tools(calendar_tools)

In [702]:
from langchain_core.messages import ToolMessage

def tool_dispatch_node(state):
    messages = state["messages"]
    last_message = messages[-1]
    tool_calls = getattr(last_message, "tool_calls", [])
    new_messages = []
    for call in tool_calls:
        tool_name = call["name"]
        args = call["args"]
        tool_map = {
            "create_event_tool": create_event_tool,
            "list_events_tool": list_events_tool,
            "postpone_event_tool": postpone_event_tool,
            "delete_event_tool": delete_event_tool,
        }
        tool_func = tool_map[tool_name]
        # For @tool-decorated functions, pass the args dict as a single argument
        result = tool_func.invoke(args)
        print("Tool name: ->>>>>>>  ", tool_name)
        print("TOol argument -->>> ",args)
        
        new_messages.append(
            ToolMessage(content=result, name=tool_name, tool_call_id=call["id"])
        )
        print(new_messages)
    return {"messages": new_messages}



In [703]:
from langgraph.graph import END

def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    if getattr(last_message, "tool_calls", None):
        return "tools"
    return END


In [714]:
from datetime import datetime
import pytz
from langchain_core.messages import ToolMessage, HumanMessage

def call_model(state):
    messages = state["messages"]

    # Import ToolMessage class if not already imported
    # from your_module import ToolMessage

    # Find the latest ToolMessage
    latest_tool_message = None
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage):
            latest_tool_message = msg
            break

    # Find the latest HumanMessage
    latest_human_message = None
    for msg in reversed(messages):
        if isinstance(msg, HumanMessage):
            latest_human_message = msg
            break

    if latest_tool_message:
        # Feed both the tool message and human message to the LLM
        prompt = (
            f"Give final response based on this tool message: {latest_tool_message}. "
            f"And also consider the user's original message: {latest_human_message}. "
            "This response created by you will be final and will be prompted to user."
        )
        response = llm.invoke(prompt)
        return {"messages": [response]}

    # Add current time information for the LLM
    timezone = pytz.timezone('Asia/Kolkata')
    current_time = datetime.now(timezone)
    current_time_str = current_time.strftime('%Y-%m-%d %H:%M:%S %Z')
    time_prompt = (
        f"The current date and time is {current_time_str}. "
        "Please consider this information when generating your response."
    )

    # Combine the time prompt with the original messages
    # You can prepend or append this to the user message or system prompt as appropriate
    if messages:
        # Example: prepend to the last user message
        last_message = messages[-1]
        if hasattr(last_message, "content"):
            last_message.content = f"{time_prompt}\n\n{last_message.content}"

    response = model_with_tools.invoke(messages)
    return {"messages": [response]}



In [715]:
from typing import Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage


class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [716]:
from langgraph.graph import StateGraph, MessagesState, START

builder = StateGraph(GraphState)

builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_node("tools", tool_dispatch_node)
builder.add_edge(START, "call_model")
builder.add_conditional_edges("call_model", should_continue, ["tools", END])
builder.add_edge("tools", "call_model")
graph = builder.compile()


In [717]:
result = graph.invoke({
    "messages": [
        {"role": "user", "content": "book a meeting tommorow 9 am"}
    ]
})


Tool name: ->>>>>>>   create_event_tool
TOol argument -->>>  {'description': '', 'end_datetime': '2025-07-08T10:00:00', 'location': '', 'start_datetime': '2025-07-08T09:00:00', 'summary': 'Meeting', 'timezone': 'Asia/Kolkata'}
[ToolMessage(content='Event created: https://www.google.com/calendar/event?eid=bHJtdnM5ajZsbDZzZzZhcXA4c2VxaTU5aWsgNjAyODNiYjAzMTU1OTY4MTQ1YWQ2OWFkYmRiOTg5MWFiNTQ3MjBmZjc1MDliNDQ2ODVlYzA4ODExMmFiNWJiMkBn', name='create_event_tool', tool_call_id='call_yw690mnexdgm75mend4tyb86')]


In [718]:
result

{'messages': [HumanMessage(content='The current date and time is 2025-07-07 09:11:50 IST. Please consider this information when generating your response.\n\nbook a meeting tommorow 9 am', additional_kwargs={}, response_metadata={}, id='6f22479d-c71a-4761-90e0-cbafbcc695d1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yw690mnexdgm75mend4tyb86', 'function': {'arguments': '{"description":"","end_datetime":"2025-07-08T10:00:00","location":"","start_datetime":"2025-07-08T09:00:00","summary":"Meeting","timezone":"Asia/Kolkata"}', 'name': 'create_event_tool'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 1068, 'total_tokens': 1144, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'system_fingerprint': None, 'id': 'o24qwEz-3NKUce-95b454126b7d5957', 'service_tier': None, 'finish_reason'

In [ ]:
ai_messages = [msg for msg in result['messages'] if isinstance(msg, AIMessage)]
if ai_messages:
    print(ai_messages[0])

In [466]:
tool_name = create_event_tool
tool_Args = {'description': '', 'end_datetime': '2024-03-07T22:00:00', 'location': '', 'start_datetime': '2024-03-07T21:00:00', 'summary': 'Meeting', 'timezone': 'UTC'}

In [467]:
tool_name.invoke(tool_Args)

'Event created: https://www.google.com/calendar/event?eid=ZWczNzE0MzU4dW5qcnA0dXZxNmlxZDI1cDggNjAyODNiYjAzMTU1OTY4MTQ1YWQ2OWFkYmRiOTg5MWFiNTQ3MjBmZjc1MDliNDQ2ODVlYzA4ODExMmFiNWJiMkBn'

In [454]:
from langchain_core.messages import AIMessage

class FakeMessage:
    def __init__(self, tool_calls):
        self.tool_calls = tool_calls

state = {
    "messages": [
        {"role": "user", "content": "Postpone my meeting"},
        AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0ba9mfbnebjvow1atlehpm8f', 'function': {'arguments': '{"end_datetime":"2024-03-15T00:00:00","match_summary":"meeting","new_end_datetime":"2024-03-14T02:00:00","new_start_datetime":"2024-03-14T01:00:00","start_datetime":"2024-03-14T00:00:00"}', 'name': 'postpone_event_tool'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 1008, 'total_tokens': 1108, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'system_fingerprint': None, 'id': 'o1z4aLH-3NKUce-95af2ae3e817549b', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--941bc2c3-ba82-40ac-b40b-228d322f6ce6-0', tool_calls=[{'name': 'postpone_event_tool', 'args': {'end_datetime': '2024-03-15T00:00:00', 'match_summary': 'meeting', 'new_end_datetime': '2024-03-14T02:00:00', 'new_start_datetime': '2024-03-14T01:00:00', 'start_datetime': '2024-03-14T00:00:00'}, 'id': 'call_0ba9mfbnebjvow1atlehpm8f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1008, 'output_tokens': 100, 'total_tokens': 1108, 'input_token_details': {}, 'output_token_details': {}})
    ]
}


In [455]:
result = tool_dispatch_node(state)
print(result)


this is toolfunc
name='postpone_event_tool' description='Postpone (reschedule) all Google Calendar events matching a summary and time window.\n\nArgs:\n    start_datetime (str): Start datetime for search window.\n    end_datetime (str): End datetime for search window.\n    match_summary (str): Substring to match event summary (case-insensitive).\n    new_start_datetime (str): New start datetime for the event(s).\n    new_end_datetime (str): New end datetime for the event(s).\n    timezone (str): Timezone.\n\nReturns:\n    str: Confirmation message or error.' args_schema=<class 'langchain_core.utils.pydantic.postpone_event_tool'> func=<function postpone_event_tool at 0x00000292BF286700>
this is arg:  {'end_datetime': '2024-03-15T00:00:00', 'match_summary': 'meeting', 'new_end_datetime': '2024-03-14T02:00:00', 'new_start_datetime': '2024-03-14T01:00:00', 'start_datetime': '2024-03-14T00:00:00'}
{'messages': [{'role': 'user', 'content': 'Postpone my meeting'}, AIMessage(content='', additi

In [ ]:
from typing import TypedDict, cast
from langchain.tools import tool

@tool
def postpone_event_tool(
    start_datetime: str,
    end_datetime: str,
    user_query: str,
    new_start_datetime: str,
    new_end_datetime: str,
    llm=None  # Pass your LLM instance here
) -> str:
    """
    Postpone a Google Calendar event based on a natural language user query.
    Uses an LLM to select the correct event if the reference is ambiguous.

    Args:
        start_datetime (str): Start datetime for search window.
        end_datetime (str): End datetime for search window.
        user_query (str): The user's original query or meeting description.
        new_start_datetime (str): New start datetime for the event.
        new_end_datetime (str): New end datetime for the event.
        llm: An LLM instance capable of function-calling or text completion.

    Returns:
        str: Confirmation message or clarification prompt.
    """ 
    timezone = "Asia/Kolkata"
    args = {
        "start_datetime": start_datetime,
        "end_datetime": end_datetime
    }
    events = list_events_tool.invoke(args)
    if not events:
        return "No events found in the specified time window."

    # Prepare event options for the LLM
    event_options = [
        f"{idx+1}. {e.get('summary', 'No Title')} at {e.get('start')} (ID: {e.get('id')})"
        for idx, e in enumerate(events)
    ]
    options_text = "\n".join(event_options)


    # Compose LLM prompt
    prompt = (
        f"User query: '{user_query}'\n"
        f"Here are the events found:\n{options_text}\n"
        "Based on the user's query, which event ID best matches the intent? "
        "Just reply with the event ID."
    )

    # Call the LLM to select the best event
    class output(TypedDict):
        event_ids = list[str]

    llm_response = cast(output,llm.with_structured_output(output).invoke(prompt))
    selected_event_id = llm_response

    # Find the event by ID
    event = next((e for e in events if e.get('id') in selected_event_id), None)
    if not event:
        return (
            "Sorry, I couldn't confidently match your request to a specific event. "
            "Please clarify or specify the event ID."
        )

    # Postpone the event
    tool = PostponeGoogleCalendarEvent(api_resource)
    result = tool._run(
        event_id=str(event.get('id')),
        new_start_datetime=new_start_datetime,
        new_end_datetime=new_end_datetime,
        timezone=timezone
    )
    return f"Postponed event: {event.get('summary')} ({selected_event_id}): {result}"


In [585]:
# Current date: 2025-07-06

start_datetime = "2024-03-16T23:00:00"      # Start of today
end_datetime = "2024-03-16T23:59:00"        # End of today
user_query = "my"           # User's natural language reference

# New times for postponement
new_start_datetime = "2024-03-17T04:00:00"  # Tomorrow at 4:00 AM
new_end_datetime = "2024-03-17T05:00:00"    # (Example: 1 hour meeting)


In [586]:
args = {
    "start_datetime": start_datetime,
    "end_datetime": end_datetime,
    "user_query": user_query,
    "new_start_datetime": new_start_datetime,
    "new_end_datetime": new_end_datetime,
    "llm": llm
}
result = postpone_event_tool.invoke(args)
print(result)


function stated
DFdfasdf
These are events
[{'start': '2024/03/16 23:00:00', 'end': '2024/03/16 23:59:59', 'summary': 'Meeting', 'description': None, 'location': None, 'hangoutLink': None, 'id': '8upev5qn9mds3i28dmk0dr3500'}]
1. Meeting at 2024/03/16 23:00:00 (ID: 8upev5qn9mds3i28dmk0dr3500)
Below is selected event ids ******
{'8upev5qn9mds3i28dmk0dr3500': '8upev5qn9mds3i28dmk0dr3500'}
this is event ->
{'start': '2024/03/16 23:00:00', 'end': '2024/03/16 23:59:59', 'summary': 'Meeting', 'description': None, 'location': None, 'hangoutLink': None, 'id': '8upev5qn9mds3i28dmk0dr3500'}
8upev5qn9mds3i28dmk0dr3500
this is inside the class and -----
8upev5qn9mds3i28dmk0dr3500
Postponed event: Meeting ({'8upev5qn9mds3i28dmk0dr3500': '8upev5qn9mds3i28dmk0dr3500'}): Event postponed: https://www.google.com/calendar/event?eid=OHVwZXY1cW45bWRzM2kyOGRtazBkcjM1MDAgNjAyODNiYjAzMTU1OTY4MTQ1YWQ2OWFkYmRiOTg5MWFiNTQ3MjBmZjc1MDliNDQ2ODVlYzA4ODExMmFiNWJiMkBn
